## Setting Up Your Python Environment

In [1]:
# %%capture
# # Install PyTorch with CUDA
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# # Install additional dependencies
# !pip install pandas nobuco tensorflowjs

# # Install utility packages
# !pip install cjm_yolox_pytorch

## Importing the Required Dependencies

In [2]:
# Import Python Standard Library dependencies
import json
from pathlib import Path

# Import YOLOX package
from cjm_yolox_pytorch.model import build_model
from cjm_yolox_pytorch.inference import YOLOXInferenceWrapper

# Import the pandas package
import pandas as pd

# Import PyTorch dependencies
import torch

# Import Nobuco dependencies
from nobuco import pytorch_to_keras, ChannelOrder

# Import TensorFlow
import tensorflow as tf

# Import TensorFlow.js dependencies
from tensorflowjs import converters, quantization

2023-09-26 12:01:33.891573: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-26 12:01:33.919443: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 12:01:34.568231: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Setting Up the Project

### Set the Directory Paths

In [3]:
# The name for the project
project_name = f"pytorch-yolox-object-detector"

# The path for the project folder
project_dir = Path(f"./{project_name}/")

# Create the project directory if it does not already exist
project_dir.mkdir(parents=True, exist_ok=True)

# The path to the checkpoint folder
checkpoint_dir = Path(project_dir/f"2023-08-17_16-14-43")

pd.Series({
    "Project Directory:": project_dir,
    "Checkpoint Directory:": checkpoint_dir,
}).to_frame().style.hide(axis='columns')

Project Directory:,pytorch-yolox-object-detector
Checkpoint Directory:,pytorch-yolox-object-detector/2023-08-17_16-14-43


## Loading the Checkpoint Data

### Load the Colormap

In [4]:
# The colormap path
colormap_path = list(checkpoint_dir.glob('*colormap.json'))[0]

# Load the JSON colormap data
with open(colormap_path, 'r') as file:
        colormap_json = json.load(file)

# Convert the JSON data to a dictionary        
colormap_dict = {item['label']: item['color'] for item in colormap_json['items']}

# Extract the class names from the colormap
class_names = list(colormap_dict.keys())

# Make a copy of the colormap in integer format
int_colors = [tuple(int(c*255) for c in color) for color in colormap_dict.values()]

### Load the Normalization Statistics

In [5]:
# The normalization stats path
norm_stats_path = checkpoint_dir/'norm_stats.json'

# Read the normalization stats from the JSON file
with open(norm_stats_path, "r") as f:
    norm_stats_dict = json.load(f)

# Convert the dictionary to a tuple
norm_stats = (norm_stats_dict["mean"], norm_stats_dict["std_dev"])

# Print the mean and standard deviation
pd.DataFrame(norm_stats)

,0,1,2
0,0.5,0.5,0.5
1,1.0,1.0,1.0


### Load the Model Checkpoint

In [6]:
# The model checkpoint path
checkpoint_path = list(checkpoint_dir.glob('*.pth'))[0]

# Load the model checkpoint onto the CPU
model_checkpoint = torch.load(checkpoint_path, map_location='cpu')

### Load the Trained YOLOX Model

In [7]:
# Select the YOLOX model configuration
model_type = checkpoint_path.stem

# Create a YOLOX model with the number of output classes equal to the number of class names
model = build_model(model_type, len(class_names))

# Initialize the model with the checkpoint parameters and buffers
model.load_state_dict(model_checkpoint)

The file ./pretrained_checkpoints/yolox_tiny.pth already exists and overwrite is set to False.


<All keys matched successfully>

## Converting the Model to TensorFlow

### Prepare the Model for Inference

In [8]:
# Convert the normalization stats to tensors
mean_tensor = torch.tensor(norm_stats[0]).view(1, 3, 1, 1)
std_tensor = torch.tensor(norm_stats[1]).view(1, 3, 1, 1)

# Set the model to evaluation mode
model.eval();

# Wrap the model with preprocessing and post-processing steps
wrapped_model = YOLOXInferenceWrapper(model, 
                                      mean_tensor, 
                                      std_tensor, 
                                      scale_inp=True, # Scale input values from the rang [0,255] to [0,1]
                                      channels_last=True, # Have the model expect input in channels-last format
                                      run_box_and_prob_calculation=False # Enable or disable post-processing steps
                                     )

### Prepare the Input Tensor

In [9]:
input_tensor = torch.randn(1, 256, 256, 3)

### Convert the PyTorch Model to Keras 

In [10]:
keras_model = pytorch_to_keras(
    wrapped_model, 
    args=[input_tensor],
    inputs_channel_order=ChannelOrder.PYTORCH,
    outputs_channel_order=ChannelOrder.PYTORCH, 
)

2023-09-26 12:01:36.089900: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-26 12:01:36.090163: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/innom-dt/mambaforge/envs/pytorch-env/lib/python3.11/site-packages/nobuco/converters/validation.py:51: RuntimeWarning: [<built-in method max_pool2d of type object at 0x7f6293c028a0>|YOLOXInferenceWrapper->YOLOX->CSPDarknet->Sequential->SPPBottleneck->MaxPool2d-

Legend:
    Green — conversion successful
    Yellow — conversion imprecise
    Red — conversion failed
    Red — no converter found
    Bold — conversion applied directly
    * — subgraph reused
    Tensor — this output is not dependent on any of subgraph's input tensors
    Tensor — this input is a parameter / constant
    Tensor — this tensor is useless

 (!) Max diff 0.029130  
 I  File "/home/innom-dt/mambaforge/envs/pytorch-env/lib/python3.11/site-packages/nobuco/trace/trace.py", line 236 
 D  File "/home/innom-dt/mambaforge/envs/pytorch-env/lib/python3.11/site-packages/cjm_yolox_pytorch/inference.py", line 24  
YOLOXInferenceWrapper[cjm_yolox_pytorch.inference](float32_0<1,256,256,3>) -> float32_662<1,1344,24>
 │  __truediv__[torch.Tensor](float32_0<1,256,256,3>, 255.0) -> float32_1<1,256,256,3>
 │  permute[torch.Tensor](float32_1<1,256,256,3>, 0, 3, 1, 2) -> float32_2<1,3,256,256>
 │  __sub__[torch.Tensor](float32_2<1,3,256,256>, float32_3<1,3,1,1>) -> float32_4<1,3,256,256>
 │

## Enabling Dynamic Input Dimensions

### Define New Input Dimensions

In [11]:
# Get the current input shape
input_shape = keras_model.layers[0].input_shape[0][1:]

# Make every dimension except the channel dimension dynamic
dynamic_input_shape = tuple(i if i == 3 else None for i in input_shape)

pd.Series({
    "Source Input Shape:": input_shape,
    "Dynamic Input Shape:": dynamic_input_shape,
}).to_frame().style.hide(axis='columns')

Source Input Shape:,"(256, 256, 3)"
Dynamic Input Shape:,"(None, None, 3)"


### Build Dynamic Keras Model

In [12]:
# Create a Keras tensor with the dynamic input shape
inputs = tf.keras.Input(shape=dynamic_input_shape)
# Get a Keras tensor with the dynamic output shape 
outputs = keras_model(inputs)

# Build a Keras model with dynamic input and output shapes
dynamic_model = tf.keras.Model(inputs, outputs)
# Add the trained weights to the dynamic Keras model
dynamic_model.set_weights(keras_model.get_weights())

### Save the Keras Model in SavedModel format

In [13]:
# Set the folder path for the SavedModel files
savedmodel_dir = Path(f"{checkpoint_dir}/{colormap_path.stem.removesuffix('-colormap')}-{model_type}-tf")
# Save the TensorFlow model to disk
dynamic_model.save(savedmodel_dir, save_format="tf")

INFO:tensorflow:Assets written to: pytorch-yolox-object-detector/2023-08-17_16-14-43/hagrid-sample-30k-384p-yolox_tiny-tf/assets


INFO:tensorflow:Assets written to: pytorch-yolox-object-detector/2023-08-17_16-14-43/hagrid-sample-30k-384p-yolox_tiny-tf/assets


## Exporting the Model to TensorFlow.js

In [14]:
# Set the path for TensorFlow.js model files
tfjs_model_dir = f"{savedmodel_dir}js-uint8"

# Convert the TensorFlow SavedModel to a TensorFlow.js Graph model
converters.convert_tf_saved_model(saved_model_dir=str(savedmodel_dir), 
                                  output_dir=tfjs_model_dir, 
                                  quantization_dtype_map={quantization.QUANTIZATION_DTYPE_UINT8:True}
                                 )

2023-09-26 12:02:48.644820: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-26 12:02:48.644926: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-09-26 12:02:48.645059: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-09-26 12:02:48.645252: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-26 12:02:48.645310: W tensorflow/core/common_runtime/gpu/gpu_d